코드의 목적: French Website에 있는 여러 Factor들을 Monthly 단위로 가져와서,

특정 반기에 premium이 큰 factor가 있는지 확인해보는 작업

Time-Series 분석의 경우 훨씬 더 deep dive 해야 하기 때문에

이 코드를 그대로 믿고 투자할 것은 아니며,

그냥 rough 한 결과 확인 용도

### 0. Import Module

In [ ]:
from google.colab import drive
import pandas as pd
import statsmodels.api as sm
import numpy as np
import os
import statsmodels.api as sm
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### 1. About Data

각각의 폴더에 각각의 데이터가 들어가있는 형태

출처: https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html#Research

2023년 7월 3일 스냅샷이며, 추후에 (과거 데이터의 약간의 수정 포함) 바뀔 수 있음

In [ ]:
list_dir_data = os.listdir("drive/MyDrive/Colab Notebooks/data/FF_230703/")
print(list_dir_data)

['Asia_Pacific_ex_Japan_MOM_Factor_CSV', 'Developed_ex_US_5_Factors_CSV', 'Developed_5_Factors_CSV', 'Developed_Mom_Factor_CSV', 'Emerging_5_Factors_CSV', 'Emerging_MOM_Factor_CSV', 'Europe_Mom_Factor_CSV', 'Europe_5_Factors_CSV', 'Developed_ex_US_Mom_Factor_CSV', 'Asia_Pacific_ex_Japan_5_Factors_CSV', 'North_America_Mom_Factor_CSV', 'F-F_Research_Data_5_Factors_2x3_CSV', 'Japan_Mom_Factor_CSV', 'F-F_Momentum_Factor_CSV', 'North_America_5_Factors_CSV', 'Japan_5_Factors_CSV']


### 2. Aggregate Data

In [91]:
def load_ff_data(dir_data_base): #dir_data_base가 폴더명
    dir_data = "drive/MyDrive/Colab Notebooks/data/FF_230703/"+dir_data_base+"/"
    assert len(os.listdir(dir_data))==1 #한 폴더 내에 csv 파일이 하나인지 체크
    dir_data += os.listdir(dir_data)[0]
    if "F-F_Momentum" in dir_data: df = pd.read_csv(dir_data, header=11) #F-F Momentum 만 구조가 다름.
    else: df = pd.read_csv(dir_data, header=2)
    df = df.rename(columns={"Unnamed: 0": "YYYYMM"}) #첫 컬럼은 yyyymm형태며, 앞뒤로 공백 있을 때도 있음. 또한 연간 데이터랑 섞여있기도 함.
    df = df[df["YYYYMM"].apply(lambda x: len(str(x).strip())==6)].copy() #따라서 월간 데이터만 놔두기 위한 작업
    df["YYYYMM"] = df["YYYYMM"].astype(int)
    df["YEAR"] = df["YYYYMM"]//100
    df["MONTH"] = df["YYYYMM"]%100
    df = df.drop(columns=["YYYYMM"]+[col for col in df.columns if col=="RF"]).sort_values(["YEAR","MONTH"]).reset_index(drop=True)
    assert (df.iloc[-1]["YEAR"]*12+df.iloc[-1]["MONTH"])-(df.iloc[0]["YEAR"]*12+df.iloc[0]["MONTH"]) + 1 == len(df) #비어있는 데이터 있지 않은지 간단한 체크
    df = df.set_index(["YEAR","MONTH"]).stack().reset_index()
    df.columns = ["YEAR", "MONTH", "TYPE", "VALUE"]
    df["TYPE"] += ("_"+dir_data_base) #컬럼명에 출처 파일명 붙여주기.
    return df
dfAgg = pd.concat([load_ff_data(dir_data_base) for dir_data_base in list_dir_data])

### 3. 데이터 사후처리

In [104]:
dfAgg["VALUE"] = dfAgg["VALUE"].astype(float)
dfAgg = dfAgg[dfAgg["VALUE"]>-99].copy() #FF는 -99를 Missing의 Indicator로서 사용
dfAgg["Bias"] = 1
dfAgg["January"] = 0
dfAgg.loc[dfAgg["MONTH"]==1, "January"] = 1
dfAgg["2H"] = 0
dfAgg.loc[dfAgg["MONTH"]>=7, "2H"] = 1
dfAgg["NovApr"] = 0
dfAgg.loc[dfAgg["MONTH"]>=11, "NovApr"] = 1
dfAgg.loc[dfAgg["MONTH"]<=4, "NovApr"] = 1

In [93]:
dfAgg.groupby(["TYPE"])["VALUE"].agg(["min","max","mean","std"]) #이상한 데이터는 별로 없어보임. 통상적으로 Anomaly의 크기는 Monthly 1% 이하 수준.

,min,max,mean,std
TYPE,,,,
CMA_Asia_Pacific_ex_Japan_5_Factors_CSV,-7.67,8.46,0.302759,2.434050
CMA_Developed_5_Factors_CSV,-6.46,9.58,0.209823,1.948453
CMA_Developed_ex_US_5_Factors_CSV,-8.51,7.03,0.149570,1.776740
CMA_Emerging_5_Factors_CSV,-8.79,6.53,0.289461,1.955942
CMA_Europe_5_Factors_CSV,-7.30,8.77,0.127215,1.825627
CMA_F-F_Research_Data_5_Factors_2x3_CSV,-7.20,9.05,0.281029,2.080735
CMA_Japan_5_Factors_CSV,-12.99,7.54,0.070886,2.350615
CMA_North_America_5_Factors_CSV,-10.78,14.39,0.281595,2.729172
HML_Asia_Pacific_ex_Japan_5_Factors_CSV,-8.91,23.70,0.542380,3.049941


### 4. 간단한 OLS
통계적으로 엄밀한 방법론은 아님

하지만 크기나 유의성들을 rough하게 보는 것에는 큰 지장 없을 것으로 보임

#### 4-1. First half와 Second Half의 차이

In [97]:
df_mdl = dfAgg.groupby(["TYPE"]).apply(lambda x: sm.OLS(x["VALUE"], x[["Bias","2H"]]).fit())
df_params = df_mdl.apply(lambda x: x.params)
df_tvalues = df_mdl.apply(lambda x: x.tvalues)
df_nobs = pd.DataFrame(df_mdl.apply(lambda x: x.nobs), columns=["nobs"])

In [98]:
df_agg = pd.merge(pd.merge(df_params, df_tvalues,
                           left_index=True, right_index=True, suffixes=("_param", "_tvalue")),
                  df_nobs, left_index=True, right_index=True)

In [99]:
df_agg

,Bias_param,2H_param,Bias_tvalue,2H_tvalue,nobs
TYPE,,,,,
CMA_Asia_Pacific_ex_Japan_5_Factors_CSV,0.333147,-0.060622,1.918765,-0.247201,395.0
CMA_Developed_5_Factors_CSV,0.191523,0.036507,1.377942,0.185963,395.0
CMA_Developed_ex_US_5_Factors_CSV,0.215888,-0.132302,1.704467,-0.739540,395.0
CMA_Emerging_5_Factors_CSV,0.187514,0.203347,1.303959,1.001240,371.0
CMA_Europe_5_Factors_CSV,0.163198,-0.071784,1.253338,-0.390314,395.0
CMA_F-F_Research_Data_5_Factors_2x3_CSV,0.268440,0.025143,2.442774,0.161899,719.0
CMA_Japan_5_Factors_CSV,0.103096,-0.064258,0.614871,-0.271333,395.0
CMA_North_America_5_Factors_CSV,0.176599,0.209462,0.907735,0.762271,395.0
HML_Asia_Pacific_ex_Japan_5_Factors_CSV,0.736701,-0.387660,3.392827,-1.264026,395.0


계수의 의미: Bias는 1-6월 리턴, 2H는 7-12월 리턴과 2-6월 리턴의 차이

전체적으로 SMB의 경우 1H가 더 강하며, 2H에는 모든 케이스에 reverse됨

나머지 Anomaly의 경우 1H/2H 차이가 크게 나는 케이스는 많지 않음

WML의 경우 Japan에서는 유의하게 2H가 강하며, 나머지 국가도 2H가 (유의하지 않게) 강함

#### 4-2. First half와 Second Half의 차이 (Jaunary Effect 고려)

In [101]:
df_mdl = dfAgg.groupby(["TYPE"]).apply(lambda x: sm.OLS(x["VALUE"], x[["Bias","January","2H"]]).fit())
df_params = df_mdl.apply(lambda x: x.params)
df_tvalues = df_mdl.apply(lambda x: x.tvalues)
df_nobs = pd.DataFrame(df_mdl.apply(lambda x: x.nobs), columns=["nobs"])

In [102]:
df_agg = pd.merge(pd.merge(df_params, df_tvalues,
                           left_index=True, right_index=True, suffixes=("_param", "_tvalue")),


                  df_nobs, left_index=True, right_index=True)

In [103]:
df_agg

,Bias_param,January_param,2H_param,Bias_tvalue,January_tvalue,2H_tvalue,nobs
TYPE,,,,,,,
CMA_Asia_Pacific_ex_Japan_5_Factors_CSV,0.244573,0.528760,0.027952,1.285718,1.137678,0.108675,395.0
CMA_Developed_5_Factors_CSV,0.165122,0.157605,0.062908,1.082804,0.422999,0.305093,395.0
CMA_Developed_ex_US_5_Factors_CSV,0.143354,0.433010,-0.059768,1.033490,1.277671,-0.318671,395.0
CMA_Emerging_5_Factors_CSV,0.182857,0.027788,0.208003,1.158595,0.072073,0.974779,371.0
CMA_Europe_5_Factors_CSV,0.121707,0.247687,-0.030293,0.852285,0.709895,-0.156889,395.0
CMA_F-F_Research_Data_5_Factors_2x3_CSV,0.198896,0.416104,0.094687,1.652925,1.413697,0.581601,719.0
CMA_Japan_5_Factors_CSV,-0.065549,1.006761,0.104387,-0.358539,2.253835,0.422277,395.0
CMA_North_America_5_Factors_CSV,0.219390,-0.255451,0.166670,1.027914,-0.489859,0.577532,395.0
HML_Asia_Pacific_ex_Japan_5_Factors_CSV,0.699634,0.221275,-0.350594,2.936685,0.380139,-1.088351,395.0


계수의 의미: Bias는 2-6월 리턴, January는 1월 리턴과 2-6월 리턴의 차이, 2H는 7-12월 리턴과 2-6월 리턴의 차이

전체적으로 SMB의 경우 1H가 더 강하며, 2H에는 모든 케이스에 reverse됨

Emerging을 제외하고는 January Effect도 강하며, January Effect를 제외하고도 2H가 1H에 비해 리턴이 낮음

Mom의 경우 미국 기준으로는 1H < 2H인 이유는 January의 강한 reversal 때문이며 (-2.47% per month),

Worldwide로도 WML은 January에 마이너스인 케이스가 많으나, obs 수가 적어서인지 통계적으로 유의하지는 않음

#### 4-3. 11-4와 나머지 구간 차이 (January Effect 고려)

주식을 11월-4월에 구매하고, 나머지 기간에는 구매하지 말라는 속설이 있어서 테스트 해봄

In [106]:
df_mdl = dfAgg.groupby(["TYPE"]).apply(lambda x: sm.OLS(x["VALUE"], x[["Bias","January","NovApr"]]).fit())
df_params = df_mdl.apply(lambda x: x.params)
df_tvalues = df_mdl.apply(lambda x: x.tvalues)
df_nobs = pd.DataFrame(df_mdl.apply(lambda x: x.nobs), columns=["nobs"])

In [107]:
df_agg = pd.merge(pd.merge(df_params, df_tvalues,
                           left_index=True, right_index=True, suffixes=("_param", "_tvalue")),


                  df_nobs, left_index=True, right_index=True)

In [108]:
df_agg

,Bias_param,January_param,NovApr_param,Bias_tvalue,January_tvalue,NovApr_tvalue,nobs
TYPE,,,,,,,
CMA_Asia_Pacific_ex_Japan_5_Factors_CSV,0.130305,0.358788,0.284241,0.751934,0.773554,1.107373,395.0
CMA_Developed_5_Factors_CSV,0.147716,0.061333,0.113678,1.061940,0.164742,0.551745,395.0
CMA_Developed_ex_US_5_Factors_CSV,0.176701,0.544545,-0.144882,1.397080,1.608610,-0.773369,395.0
CMA_Emerging_5_Factors_CSV,0.359297,-0.011226,-0.137426,2.493352,-0.029111,-0.643911,371.0
CMA_Europe_5_Factors_CSV,0.197817,0.374909,-0.203332,1.520357,1.076566,-1.055056,395.0
CMA_F-F_Research_Data_5_Factors_2x3_CSV,0.144429,0.237300,0.233271,1.316780,0.807411,1.434953,719.0
CMA_Japan_5_Factors_CSV,0.124112,1.107939,-0.290839,0.745187,2.485433,-1.178944,395.0
CMA_North_America_5_Factors_CSV,0.118680,-0.575697,0.420956,0.610838,-1.107071,1.462759,395.0
HML_Asia_Pacific_ex_Japan_5_Factors_CSV,0.470964,0.368970,0.080975,2.163543,0.633288,0.251140,395.0


계수의 의미: Bias는 5-10월 리턴, January는 1월 리턴과 5-10월 리턴의 차이, NovApr는 5-10월 리턴과 11-12 & 2-4월 리턴의 차이

MKTRf가 11-12 & 2-4월에 가장 강한 것으로 보임

Mom의 경우는 US 기준 11-12 & 2-4월에 가장 강한 것으로 보이는데, US의 결과와 다른 시장의 결과가 매우 다름 (샘플 기간의 차이 일 수 있음)

smb는 여전히 January에 강하나, 5-10월과 11-12 & 2-4월의 차이는 inconclusive